## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

In [6]:
from collections import defaultdict
import itertools

def Enumerator(start=0, initial=()):
    return defaultdict(itertools.count(start).__next__, initial)

In [7]:
import glob
import csv
from tqdm.auto import tqdm

id_enumerator = Enumerator()
row_dict = {}
left_id_set = set()
right_id_set = set()
rows_total = 1081 + 1092
clusters_total = 1097

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/Abt-Buy/Abt.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'abt'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/Abt-Buy/Buy.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'buy'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [8]:
true_pair_set = set()

with open(f'{home_dir}/Downloads/Abt-Buy/abt_buy_perfectMapping.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[row['idAbt']]
        id_right = id_enumerator[row['idBuy']]
        true_pair_set.add((id_left, id_right))

len(true_pair_set)

1097

In [9]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict

cluster_mapping, cluster_dict = id_pairs_to_cluster_mapping_and_dict(true_pair_set)
len(cluster_mapping)

2173

In [10]:
len(cluster_dict)

1076

In [11]:
cluster_attr = 'cluster_id'
max_cluster_id = max(cluster_mapping.values())

for row_id, row in tqdm(row_dict.items()):
    try:
        row[cluster_attr] = cluster_mapping[row_id]
    except KeyError:
        row[cluster_attr] = max_cluster_id
        max_cluster_id += 1

In [12]:
[row_dict[row_id] for row_id in next(iter(true_pair_set))]

[{'id': 305,
  'name': 'TiVo HD Digital Video Recorder (180 Hour) - TCD652160',
  'description': 'TiVo HD Digital Video Recorder - TCD652160/ Search, Record And Watch Shows In HD/ Record Up To 20 Hours In HD (Or 180 Hours In Standard Definition)/ Record Two Shows At Once In HD/ Replaces Your Cable Box And Works With Over-The-Air Antenna/ USB Connectivity/ Remote Control/ Netflix Instant Streaming/ TiVo Service Required And Sold Separately',
  'price': '',
  'source': 'abt',
  'cluster_id': 305},
 {'id': 1508,
  'name': 'Tivo TIV652160 Tivo High Definition',
  'description': '',
  'manufacturer': 'TiVo',
  'price': '',
  'source': 'buy',
  'cluster_id': 305}]

## Preprocess

In [13]:
attr_list = ['name', 'description', 'price']

In [14]:
import unidecode
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [15]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [16]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [17]:
attr_info_dict = {
    'name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'description': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_name': {
        'source_attr': 'name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'semantic_description': {
        'source_attr': 'description',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [18]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

17:18:50 INFO:For attr=name, computing actual max_str_len
17:18:50 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:18:50 INFO:For attr=name, using actual_max_str_len=16
17:18:50 INFO:For attr=description, computing actual max_str_len
17:18:50 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:18:50 INFO:For attr=description, using actual_max_str_len=16
17:18:50 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:18:53 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:18:56 INFO:For attr=price, computing actual max_str_len
17:18:56 INFO:For attr=price, using actual_max_str_len=14


{'name': NumericalizeInfo(attr='name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'description': NumericalizeInfo(attr='description', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '

In [19]:
from entity_embed import LinkageDataModule

train_cluster_len = 200
valid_cluster_len = 200
datamodule = LinkageDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=clusters_total - valid_cluster_len - train_cluster_len,
    only_plural_clusters=True,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    random_seed=random_seed
)

## Training

In [20]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [21]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.3',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'abt-buy'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name),
)

17:18:56 INFO:GPU available: True, used: True
17:18:56 INFO:TPU available: None, using: 0 TPU cores
17:18:56 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [22]:
trainer.fit(model, datamodule)

17:18:56 INFO:Train pair count: 214
17:18:56 INFO:Valid pair count: 200
17:18:56 INFO:Test pair count: 704
17:18:58 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.9 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
5.4 M     Trainable params
2.5 M     Non-trainable params
7.9 M     Total params


1

In [23]:
model.blocker_net.get_signature_weights()

{'name': 0.26724737882614136,
 'description': 0.19677719473838806,
 'semantic_name': 0.21819987893104553,
 'semantic_description': 0.16555693745613098,
 'price': 0.15221861004829407}

In [24]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.32300163132137033,
 'valid_f1_at_0.5': 0.6321428571428571,
 'valid_f1_at_0.7': 0.6807387862796833,
 'valid_f1_at_0.9': 0.2222222222222222,
 'valid_pair_entity_ratio_at_0.3': 2.565,
 'valid_pair_entity_ratio_at_0.5': 0.9,
 'valid_pair_entity_ratio_at_0.7': 0.4475,
 'valid_pair_entity_ratio_at_0.9': 0.0625,
 'valid_precision_at_0.3': 0.19298245614035087,
 'valid_precision_at_0.5': 0.49166666666666664,
 'valid_precision_at_0.7': 0.7206703910614525,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 0.99,
 'valid_recall_at_0.5': 0.885,
 'valid_recall_at_0.7': 0.645,
 'valid_recall_at_0.9': 0.125}

## Testing

In [25]:
trainer.test(ckpt_path='best', verbose=False)

17:20:00 INFO:Train pair count: 214
17:20:00 INFO:Valid pair count: 200
17:20:00 INFO:Test pair count: 704


[{'test_f1_at_0.3': 0.10998552821997107,
  'test_f1_at_0.5': 0.32729155877783533,
  'test_f1_at_0.7': 0.534698521046644,
  'test_f1_at_0.9': 0.21777221526908633,
  'test_pair_entity_ratio_at_0.3': 8.600292825768667,
  'test_pair_entity_ratio_at_0.5': 2.322108345534407,
  'test_pair_entity_ratio_at_0.7': 0.7818448023426061,
  'test_pair_entity_ratio_at_0.9': 0.07979502196193265,
  'test_precision_at_0.3': 0.05822267620020429,
  'test_precision_at_0.5': 0.19924337957124844,
  'test_precision_at_0.7': 0.4400749063670412,
  'test_precision_at_0.9': 0.7981651376146789,
  'test_recall_at_0.3': 0.991304347826087,
  'test_recall_at_0.5': 0.9159420289855073,
  'test_recall_at_0.7': 0.6811594202898551,
  'test_recall_at_0.9': 0.12608695652173912}]

## Testing manually

In [26]:
test_row_dict = datamodule.test_row_dict
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [27]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [28]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [29]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 389 ms, sys: 25 ms, total: 414 ms
Wall time: 77.7 ms


In [30]:
%%time

sim_threshold = 0.3
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 592 ms, sys: 0 ns, total: 592 ms
Wall time: 90.4 ms


In [31]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

8.600292825768667

In [32]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.05822267620020429, 0.991304347826087)

In [33]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

11064

In [34]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

6

In [35]:
cos_similarity = lambda a, b: np.dot(a, b)

In [36]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.27090424,
 {'id': 537,
  'name': "kenwood 6 . 1 ' all - in - one double - din in - dash navigation - dnx5120",
  'description': "kenwood 6 . 1 ' all - in - one double - din in - dash navigation - dnx5120 / built - in garmin navigation board / usb direct control",
  'price': '',
  'source': 'abt',
  'cluster_id': 537},
 {'id': 1723,
  'name': 'dnx5120 all - in - one navigation / dvd entertainment system',
  'description': '',
  'manufacturer': 'Kenwood',
  'price': '$ 635 . 00',
  'source': 'buy',
  'cluster_id': 537})

(0.25075477,
 {'id': 123,
  'name': 'plantronics voyager 510 bluetooth headset - silver finish - 496915',
  'description': 'plantronics voyager 510 bluetooth headset - 496915 / lightweight design for all - day wearing comfort / noise - canceling microphone for clearer conversations / wind - noise reduction technology',
  'price': '',
  'source': 'abt',
  'cluster_id': 123},
 {'id': 1453,
  'name': 'plantronics voyager 510 bluetooth headset',
  'description': '',
  'manufacturer': 'Plantronics',
  'price': '',
  'source': 'buy',
  'cluster_id': 123})

(0.19098271,
 {'id': 807,
  'name': 'canon deluxe black digital camera case - 2595b002',
  'description': 'canon deluxe black digital camera case - 2595b002 / soft nylon case / flip - down cover / belt loop attachment for hands - free convenience / compatible with canon',
  'price': '$ 13 . 99',
  'source': 'abt',
  'cluster_id': 807},
 {'id': 1523,
  'name': 'canon psc - 85 deluxe soft case for the powershot - a650 is and a720 is - 2595b002 ( aa )',
  'description': '',
  'manufacturer': 'Canon',
  'price': '$ 8 . 77',
  'source': 'buy',
  'cluster_id': 807})

(0.28638005,
 {'id': 626,
  'name': 'polk audio csi a4 black center channel loudspeaker - csia4bk',
  'description': "polk audio csi a4 black center channel loudspeaker - csia4bk / 1 ' silk / polymer dome tweeter / dual 5 - 1 / 4 ' mid / woofers /",
  'price': '$ 279 . 95',
  'source': 'abt',
  'cluster_id': 626},
 {'id': 1552,
  'name': 'polkaudio csi a4 black high performance center channel loudspeaker - am4415 - a',
  'description': '',
  'manufacturer': 'Polkaudio',
  'price': '$ 189 . 47',
  'source': 'buy',
  'cluster_id': 626})

(0.1468496,
 {'id': 298,
  'name': 'lg pearl gray xl capacity electric dryer - dle5955g',
  'description': 'lg pearl gray xl capacity electric dryer - dle5955g / 7 . 3 cu . ft . capacity / 9 drying programs / 5 temperature levels / precise temperature control',
  'price': '',
  'source': 'abt',
  'cluster_id': 298},
 {'id': 1503,
  'name': 'lg 7 . 3 cu . ft . front control , st drum , chrome door trim ( pearl gray )',
  'description': '',
  'manufacturer': 'LG Electronics',
  'price': '',
  'source': 'buy',
  'cluster_id': 298})

(0.21003205,
 {'id': 820,
  'name': 'sony bud style headphones in black - mdrex85lpblk',
  'description': 'sony bud style headphones in black - mdrex85lpblk / 13 . 5mm drivers / super - light weight / studio monitor quality / soft silicone earbuds / gold - plated',
  'price': '',
  'source': 'abt',
  'cluster_id': 820},
 {'id': 1510,
  'name': 'sony mdrex81lp comfort fit ear hanger headphones - choose color ( black )',
  'description': '',
  'manufacturer': 'Sony',
  'price': '',
  'source': 'buy',
  'cluster_id': 820})